In [10]:
import json
import re
import pandas as pd
import unicodecsv as csv

First: load the business dataset (114.5 MB), retrieve the only two columns that are actually relevant, and save. (By saving in a spreadsheet instead of a collection of JSON objects, we get to eliminate all the space otherwise wasted by saving JSON keys.) New file size: 1.08 MB.

In [11]:
with open("yelp_dataset_challenge_round9/yelp_academic_dataset_business.json") as data_file:
        businesses_df = pd.DataFrame([json.loads(line) for line in data_file.readlines()])

In [12]:
US_States = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
             "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
             "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
             "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
             "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

businesses_df = businesses_df[businesses_df['state'].isin(US_States)]

In [13]:
businesses_df = businesses_df.loc[:, ['business_id', 'stars']]
# create a set of us_business_ids, will need for fast lookups in the review-object section
us_business_ids_set = set(businesses_df['business_id'])
# drop the index column, we won't need it saved in the CSV
businesses_df.drop(businesses_df.columns[0], axis=1, inplace=True)
businesses_df.to_csv("businesses_shortened.csv")

Next: load the review dataset (3.46 GB), retrieve the subset of reviews that are for US businesses, discard irrelevant data in the JSON review objects, clean up the text, and save each object as a line in a csv. Again, by saving in a spreadsheet, we get to eliminate all the space otherwise wasted by saving JSON keys. New size: 1.59 GB.

In [14]:
def format_text(text_string):
    return re.sub('[^0-9 a-zA-Z]+', '', text_string).lower()

with open("yelp_dataset_challenge_round9/yelp_academic_dataset_review.json") as data_file_in:
    with open("reviews_shortened.csv", "w") as csv_file_out:
        fieldnames = sorted(['user_id', 'business_id', 'stars', 'date', 'text'])
        writer = csv.DictWriter(csv_file_out, fieldnames=fieldnames, extrasaction='ignore')
        writer.writeheader()
        
        for line in data_file_in:
            review = json.loads(line)
            review['text'] = format_text(review['text'])
            if review['business_id'] in us_business_ids_set and review['text'] != '':
                writer.writerow(review)